In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# DEMO (6.30)

### 0. Hyperparams

In [2]:
# A few hyperparameters.
future_num = 3                      # Change it carefully because the order of futures also matters.
choose_num = 33                       
test_num = 10
bootsrap_num = 20
test_length = 100                   # The length of the test data
batch_size = 250                      # Batch size is important when constructing the dataloader.
hidden_size = 64                    # Neural network hyperparams
learning_rate = 1e-3                                
epochs = 5                          # Iterating times

### 1. Working with data

In [3]:
# Load the prepared data.
Price = pd.read_csv('./test_'+str(choose_num)+'/Dataset.csv', encoding='gbk')
Rate_3m = pd.read_csv('./test_'+str(choose_num)+'/Rate_3m.csv', encoding='gbk')
N = Rate_3m.shape[0]                                     
# N days counted

Price = Price.drop(Price.tail(1).index)
# Same length(PRICE & RATE)

# By considering the multiplier seperately, we can multiply Price by the 'variables' immediately.
Price[['IF','IH']] = 300 * Price[['IF','IH']]
Price[['IC']] = 200 * Price[['IC']]
Price[Price.columns.difference(['Date','IF','IC','IH'])] = 100 * Price[Price.columns.difference(['Date','IF','IC','IH'])]

# Transform data to np.array(dtype=float) and put 'date' away.
Price_np = np.array((np.array(Price))[:,1:], dtype=float)
Rate_3m_np = np.array((np.array(Rate_3m))[:,1:], dtype=float)

# PR is all we need.
PR = Price_np * Rate_3m_np
# From the perspective of 'mlp_solve.py', PR = PR_F_bst '+' PR_E_bst
# Thanks to the DataLoader from torch.utils.data, it could be easier to sample data with 'shuffle', as follows.
print('数据集PR尺寸为:{}'.format(PR.shape))

数据集PR尺寸为:(362, 36)


In [4]:
# Constructing the training dataset and the test dataset.
class hedge_data(Dataset):
    def __init__(self, data, transform=None):
        super(hedge_data, self).__init__()
        # 'data' is designded based on 'PR'. 
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.transform(self.data[index])

train_data = hedge_data(PR[:-test_length,:], transform=torch.tensor)    # test_length = the size of the test data
test_data = hedge_data(PR[-test_length:,:], transform=torch.tensor)

# Constructing dataloaders.

train_dataloader = DataLoader(train_data, batch_size=batch_size, drop_last=True, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, drop_last=True)   
# Only shuffle the training data.

### 2.Creating Models

In [5]:
# Confirm our device first.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [6]:
class naivenn(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(naivenn, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Here we can make some differences about the structure of the neural network.
        # Try CNN, MLP...

        self.naive_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU(),                         
            # We try to make the output >= 0 by applying ReLU, but still can improve.
        )
    
    def forward(self, x):
        portfolio = self.naive_stack(x.to(torch.float))
        # Split the portfolio to futures and etfs.
        holding_f, holding_e = portfolio.split([future_num, portfolio.shape[1]-future_num], dim=1)  
        return holding_f, holding_e                    # Output is the portfolio.

# Now we can bulid our naive model!

naive_model = naivenn(PR.shape[1], hidden_size, PR.shape[1])
loss_f = nn.MSELoss()
opt = torch.optim.SGD(naive_model.parameters(), lr = learning_rate)


### 3. Training and testing

In [7]:
# Train:
def trainloop(dataloader, model, loss_function, optimizer):
    # Running trainloop once means that we 'bianli' or traverse the dataloader once.
    for idx, x in enumerate(dataloader):
        # Compute loss
        hold_f, hold_e = model(x)
        hold_f, hold_e = torch.tensor(hold_f, dtype=float), torch.tensor(hold_e, dtype=float)
        # The above line will raise a warning, but we can ignore it.
        # Mind the dtype.
        # Split the PnL data.
        pr_f, pr_e = x.split([future_num, hold_e.shape[1] + hold_f.shape[1] - future_num], dim=1) 

        error = -(hold_f @ pr_f.T - hold_e @ pr_e.T)              # PnL error               
        loss = loss_function(error, torch.zeros_like(error))      # loss
        # Backpropagation
        optimizer.zero_grad()            # Set the gradient as 0.
        loss = loss.requires_grad_()
        loss.backward()                  # Compute the gradient by BP algorithm.
        optimizer.step()                 # Update the parameters with the given optimizer.

# Test:
def testloop(dataloader, model, loss_function):
    # Running testloop once we will obtain a list of error rate, then we observe the quantile and minimum.
    test_loss = []
    # test_loss = list[abs(error_i) / cost_i]

    with torch.no_grad():
        for idx, x in enumerate(dataloader):
            hold_f, hold_e = model(x)
            hold_f, hold_e = np.array(hold_f, dtype=float), np.array(hold_e, dtype=float)
            idx_cost = len(train_data) + idx
            cost_array = Price_np[idx_cost]
            cost_f = cost_array[:future_num]
            cost_e = cost_array[future_num:]
            rate_array = Rate_3m_np[idx_cost]
            rate_f = rate_array[:future_num]
            rate_e = rate_array[future_num:]

            hc_f, hc_e = hold_f * cost_f.T, hold_e * cost_e.T
            # hc = hold * cost
            # hc_f or hc_e may be zero!


            # Before we compute the rate error, we should scale the hc_f and hc_e!
            if np.sum(hc_f != 0) and np.sum(hc_e != 0):

                hc_f = hc_f / np.sum(hc_f)    
                hc_e = hc_e / np.sum(hc_e)

                Rate_F = hc_f @ rate_f.T
                Rate_E = hc_e @ rate_e.T
                test_loss.append(Rate_E - Rate_F)
    
    test_loss_np = np.array(test_loss)
    # print the min and the quantile.
    print('test: min:{}, quantile:{}'.format(np.min(test_loss_np), np.percentile(test_loss_np, 5, interpolation='midpoint')))
        
        



In [8]:
for i in range(epochs):
    print(f'Epoch{i+1}\n-----------------------')
    trainloop(train_dataloader, naive_model, loss_f, opt)
    testloop(test_dataloader, naive_model, loss_f)
    
        


Epoch1
-----------------------


C:\Users\123\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


test: min:-0.02444471391362239, quantile:-0.008711938767885567
Epoch2
-----------------------
test: min:-0.02444471391362239, quantile:-0.008711938767885567
Epoch3
-----------------------
test: min:-0.02444471391362239, quantile:-0.008711938767885567
Epoch4
-----------------------
test: min:-0.02444471391362239, quantile:-0.008711938767885567
Epoch5
-----------------------
test: min:-0.02444471391362239, quantile:-0.008711938767885567
